## Explore Squad v1.1

In [1]:
%ls



explore_squad_v1_1.ipynb


In [27]:
import os, sys
path_project = '~/fontes/exqa-complearning/'

sys.path.append(os.path.abspath('.'))
sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath(r'../..'))
print(sys.path)

['/home/borela/fontes/exqa-complearning/source/exploration', '/home/borela/miniconda3/envs/relevar-busca/lib/python37.zip', '/home/borela/miniconda3/envs/relevar-busca/lib/python3.7', '/home/borela/miniconda3/envs/relevar-busca/lib/python3.7/lib-dynload', '', '/home/borela/miniconda3/envs/relevar-busca/lib/python3.7/site-packages', '/home/borela/miniconda3/envs/relevar-busca/lib/python3.7/site-packages/IPython/extensions', '/home/borela/.ipython', '/home/borela/fontes/exqa-complearning/source/exploration', '/home/borela/fontes/exqa-complearning/source/exploration/.\\.', '/home/borela/fontes/exqa-complearning/source/exploration', '/home/borela/fontes/exqa-complearning/source/exploration', '/home/borela/fontes/exqa-complearning/source/exploration', '/home/borela/fontes/exqa-complearning/source/exploration', '/home/borela/fontes/exqa-complearning/source/exploration', '/home/borela/fontes/exqa-complearning/source/exploration/.\\.', '/home/borela/fontes/exqa-complearning/source/exploration'

In [23]:
%cd ..


/home/borela/fontes/exqa-complearning


In [28]:
%pwd

'/home/borela/fontes/exqa-complearning'

In [30]:
from transformers.data.processors.squad import SquadV1Processor # SquadV2Processor

processor = SquadV1Processor()
examples = processor.get_dev_examples("data/dataset/squad/", filename="dev-v1.1.json")
print(len(examples))

100%|██████████| 48/48 [00:02<00:00, 23.14it/s]

10570


In [31]:
def display_example(qid):    
    from pprint import pprint

    idx = qid_to_example_index[qid]
    q = examples[idx].question_text
    c = examples[idx].context_text
    a = [answer['text'] for answer in examples[idx].answers]
    
    print(f'Example {idx} of {len(examples)}\n---------------------')
    print(f"Q: {q}\n")
    print("Context:")
    pprint(c)
    print(f"\nTrue Answers:\n{a}")



In [32]:
    
# generate some maps to help us identify examples of interest
qid_to_example_index = {example.qas_id: i for i, example in enumerate(examples)}
qid_to_has_answer = {example.qas_id: bool(example.answers) for example in examples}
answer_qids = [qas_id for qas_id, has_answer in qid_to_has_answer.items() if has_answer]
# no_answer_qids = [qas_id for qas_id, has_answer in qid_to_has_answer.items() if not has_answer]


In [33]:
display_example(answer_qids[1300])


Example 1300 of 10570
---------------------
Q: Where was the  Continental Edison Company located?

Context:
('In 1882, Tesla began working for the Continental Edison Company in France, '
 'designing and making improvements to electrical equipment. In June 1884, he '
 'relocated to New York City:57–60 where he was hired by Thomas Edison to work '
 "at his Edison Machine Works on Manhattan's lower east side. Tesla's work for "
 'Edison began with simple electrical engineering and quickly progressed to '
 'solving more difficult problems.')

True Answers:
['France', 'in France', 'France']


[]

## Config ...